In [11]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [12]:
len(tf.config.list_physical_devices('GPU'))

1

In [13]:
def process_csv_file(file):
    try:
     csv_data = pd.read_csv(file)
    except:
         print("CANT READ CSV: ", file)
         return
    
    if csv_data.isnull().values.any():
            
            return False
    try:
        csv_data = csv_data.drop("class", axis = 1)
    except KeyError:
        pass
    return (csv_data.to_numpy())

In [4]:
!cd /Tmp/ASL-data/Database-augmented/coffee/; ls;
!ls

/bin/bash: line 1: cd: /Tmp/ASL-data/Database-augmented/coffee/: No such file or directory
 american-sign-language_1		   modelCNN.ipynb
'ASL Converter.code-workspace'		   modelYOLO.ipynb
 ASL_Language_translation.code-workspace   MS-ASL
 CNN_ASL_letters_1.h5			   NLP
 CNN_models				   README.md
 data_augmentation			   Report.docx
 frontend				   requirements.txt
 gitignore.txt				   tempCodeRunnerFile.python
 IMAGE_vision				   training_models
 main.py				   XINLEI_L_ALY_S_MRSTF_REPORT.docx
 american-sign-language_1		   modelCNN.ipynb
'ASL Converter.code-workspace'		   modelYOLO.ipynb
 ASL_Language_translation.code-workspace   MS-ASL
 CNN_ASL_letters_1.h5			   NLP
 CNN_models				   README.md
 data_augmentation			   Report.docx
 frontend				   requirements.txt
 gitignore.txt				   tempCodeRunnerFile.python
 IMAGE_vision				   training_models
 main.py				   XINLEI_L_ALY_S_MRSTF_REPORT.docx


In [14]:
file = "/Tmp/ASL-data/Database-augmented/test/coffee/2sGQuduhAf41354.csv"
process_csv_file(file)

array([[ 0.51505029,  0.337479  , -0.79841799, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5149141 ,  0.33766055, -0.79397482, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.51484638,  0.33796486, -0.76542687, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.52175295,  0.3408646 , -0.72429359, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.52186459,  0.34013429, -0.74814087, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.52185893,  0.33947977, -0.76047277, ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [18]:
def prepare_data(folder_path, labels, train=True):
    total = 0
    max_length = 211
    
    # find max length
    # for folder in os.listdir(folder_path):
    #     print(folder)
    #     if folder != ".DS_Store":
    #         total += len(os.listdir(os.path.join(folder_path, folder)))
    #         for file in os.listdir(os.path.join(folder_path, folder)):
    #             if file != ".DS_Store":
    #                 data = process_csv_file(os.path.join(folder_path, folder, file))
    #                 if data is not(False):
    #                     if data.shape[0] > max_length:
    #                         max_length = data.shape[0]
    # print(max_length)
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    print("----------")
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            
            i=0
            for file in os.listdir(os.path.join(folder_path, folder)):
                if file != ".DS_Store": 
                    data = process_csv_file(os.path.join(folder_path, folder, file))
                    if data is not(False):
                        data = set_padding(data, max_length)
                        X = np.append(X, data, axis=0)
                        Y = np.append(Y, labels[folder])
                        if (i% 50 == 0):
                            print(i)
                i += 1
          
    print(total, max_length)
    return X, Y

In [19]:
folder_path_train = "/Tmp/ASL-data/Database-augmented/train"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_train, Y_train = prepare_data(folder_path_train, labels)

----------
door
0
50
100
150
200
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
coffee
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
milk
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
dog
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700

In [20]:
folder_path_test = "/Tmp/ASL-data/Database-augmented/test"
labels = {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_test, Y_test = prepare_data(folder_path_test, labels, train=False)

----------
door
0
50
100
150
200
250
300
350
400
450
coffee
0
50
100
150
200
250
300
350
400
450
milk
0
50
100
150
200
250
300
350
400
450
dog
0
50
100
150
200
250
300
350
0 211


In [13]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [24]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [21]:
# shuffle data
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_val,Y_val = shuffle(X_test, Y_test, random_state=0)
X_train, Y_train

(array([[[-0.30837396, -0.13019815, -0.65601784, ..., -0.4046454 ,
           0.        ,  0.        ],
         [-0.30789436, -0.13073483, -0.67463034, ..., -0.4052157 ,
           0.        ,  0.        ],
         [-0.3084117 , -0.13114377, -0.68334162, ..., -0.40571374,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],
 
        [[-0.26039057, -0.20227367, -0.73990929, ..., -0.32359282,
           0.        ,  0.        ],
         [-0.26080032, -0.20570173, -0.85264206, ..., -0.32754375,
           0.        ,  0.        ],
         [-0.26098443, -0.20697016, -0.85871232, ..., -0.32930087,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.       

In [22]:
X_train.shape, Y_train.shape

((11726, 211, 300), (11726,))

In [23]:
x_shape = X_train.shape

In [87]:
from tensorflow.keras import layers
masking_input = []
for i in range(x_shape[0]):
    masking_input.append(X_train[i].T[0])

masking_input = np.array(masking_input)

masking_input.shape

(11060, 211)

In [20]:
embedding = layers.Embedding(input_dim=x_shape[1], output_dim=x_shape[1], mask_zero=True)
masked_output = embedding(masking_input)

print(masked_output._keras_mask)
masked_output
# masking_layer = layers.Masking()
# # Simulate the embedding lookup by expanding the 2D input to 3D,
# # with embedding dimension of 10.
# unmasked_embedding = tf.cast(
#     tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
# )

# masked_embedding = masking_layer(unmasked_embedding)
# print(masked_embedding._keras_mask)

2023-04-21 00:13:36.221629: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-21 00:13:36.222041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(457, 211), dtype=bool)


<tf.Tensor: shape=(457, 211, 211), dtype=float32, numpy=
array([[[ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        ...,
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526]],

       [[ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        ..

In [49]:
#LSTM model 
from tensorflow.keras import layers
x_shape = (457, 211, 300)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
import keras

model_lstm = Sequential()
model_lstm.add(LSTM(512, return_sequences=True, input_shape=(211, 300)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(256, return_sequences=True))
model_lstm.add(Dropout(0.2))

model_lstm.add(LSTM(64,return_sequences=True)) # returns the previous sequences (e.g. long-term memory)
model_lstm.add(Flatten())
model_lstm.add(Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(0.01)))
model_lstm.add(Dropout(0.25))
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(4, activation='softmax'))


2023-06-17 17:51:46.151073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-17 17:51:46.151941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-17 17:51:46.152540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [50]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "/Tmp/shariffa/LSTM-4-labels-FULL-DATA.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [51]:
model_lstm.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 211, 512)          1665024   
                                                                 
 dropout_35 (Dropout)        (None, 211, 512)          0         
                                                                 
 lstm_33 (LSTM)              (None, 211, 256)          787456    
                                                                 
 dropout_36 (Dropout)        (None, 211, 256)          0         
                                                                 
 lstm_34 (LSTM)              (None, 211, 64)           82176     
                                                                 
 flatten_8 (Flatten)         (None, 13504)             0         
                                                                 
 dense_33 (Dense)            (None, 256)              

In [52]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model_lstm.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val), batch_size=0,callbacks=[checkpoint])

2023-06-17 17:52:05.483749: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.76GiB (rounded to 2969023232)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-06-17 17:52:05.483786: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-06-17 17:52:05.483797: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 122, Chunks in use: 122. 30.5KiB allocated for chunks. 30.5KiB in use in bin. 5.6KiB client-requested in use in bin.
2023-06-17 17:52:05.483804: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 9, Chunks in use: 9. 5.2KiB allocated for chunks. 5.2KiB in use in bin. 4.5KiB client-requested in use in bin.
2023-06-17 17:52:05.483811: I tensorflow/tsl/framewor

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

 17:52:05.484024: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512030c00 of size 65536 next 15
2023-06-17 17:52:05.484029: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512040c00 of size 256 next 24
2023-06-17 17:52:05.484033: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512040d00 of size 256 next 25
2023-06-17 17:52:05.484037: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512040e00 of size 512 next 30
2023-06-17 17:52:05.484042: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512041000 of size 256 next 27
2023-06-17 17:52:05.484046: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512041100 of size 256 next 21
2023-06-17 17:52:05.484051: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512041200 of size 2048 next 22
2023-06-17 17:52:05.484055: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fe512041a00 of size 1024 next 28
2023-06-17 17:52:05.484060: I tensorflow/tsl/f

In [ ]:
inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.GRU(64)(inputs, mask=masked_output._keras_mask)
# x = layers.LSTM(64)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model_lstm2= tf.keras.Model(inputs, outputs)
model_lstm2.summary()

In [ ]:
model_lstm2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)